In [75]:
import pandas as pd
from collections import defaultdict
import numpy as np
import sys


proteomicsDataColumns = ["Majority protein IDs", 
                        "Gene names", "Q-value", 
                        "Score", 
                        "Intensity \d+_\d+", 
                        "LFQ intensity \d+_\d+", 
                        "Glycation site positions", 
                        "Glycation site positions",
                        "Reverse",
                        "Potential contaminant",
                        "Only identified by site"]

proteomicsDataFilters = ["Reverse",
                       "Potential contaminant",
                       "Only identified by site"]

proteomicsDataProteinCol = "Majority protein IDs"

proteomicsDataGeneCol = "Gene names"
proteomicsDataModPosCol = ["Glycation site positions", "Oxidation (M) site positions"]
proteomicsDataLog = "log10"

def loadProteomicsDataset(uri):
    ''' This function gets the molecular data from a proteomics experiment.
        Input: uri of the processed file resulting from MQ
        Output: pandas DataFrame with the columns and filters defined in config.py '''

    aux = None
    
    #Get the columns from config and divide them into simple or regex columns
    columns = proteomicsDataColumns
    regexCols = [c for c in columns if '+' in c]
    columns = set(columns).difference(regexCols)
        
    #Read the filters defined in config, i.e. reverse, contaminant, etc.
    filters = proteomicsDataFilters
    proteinCol = proteomicsDataProteinCol
    geneCol = proteomicsDataGeneCol
    modPosCol = proteomicsDataModPosCol
    
    
    #Read the data from file
    data = pd.read_csv(uri, sep = '\t', low_memory=False)

    #Apply filters
    data = data[data[filters].isnull().any(1)]
    data = data.drop(filters, axis=1)
    
    #Select first protein form group, i.e. P01911;Q29830;Q9MXZ4 -> P01911
    #Set protein as index
    proteins = data[proteinCol].str.split(';').apply(pd.Series,1)[0]
    data[proteinCol] = proteins
    data = data.set_index(proteinCol)
    filters.append(proteinCol)
    columns = set(columns).difference(filters)
    
    #As well, select first gene name and first modification position corresponding
    #to the selected protein
    genes = data[geneCol].str.split(';').apply(pd.Series,1)[0]
    data[geneCol] = genes
    for modCol in modPosCol:
        mod = data[modCol].str.split(';').apply(pd.Series,1)[0]
        data[modCol] = mod

    #Get columns using regex
    for regex in regexCols:
        if aux is None:
            aux = data.filter(regex=regex)
        else:
            aux = aux.join(data.filter(regex=regex))

    #Add simple and regex columns into a single DataFrame
    data = data[list(columns)]
    data = data.join(aux)

    return data, regexCols

def extractSubjectReplicates(data, regex):
    subjectDict = defaultdict(list)
    for r in regex:
        columns = data.filter(regex = r).columns
        for c in columns:
            fields  = c.split('_')
            value = " ".join(fields[0].split(' ')[0:-1])
            subject = fields[1]
            ident = value + " " + subject 
            subjectDict[ident].append(c)

    return subjectDict
            
def parseProteomicsDataset(projectID, uri):
    log = proteomicsDataLog
    data, regex = loadProteomicsDataset(uri)
    subjectDict = extractSubjectReplicates(data, regex)
    delCols = []
    for subject in subjectDict:
            delCols.extend(subjectDict[subject])
            aux = data[subjectDict[subject]]
            data[subject] = calculateMedianReplicates(aux, log)

    data = data.drop(delCols, 1)
    
    
    return data

def calculateMedianReplicates(data, log = "log2"):
    if log == "log2":
        data = data.applymap(lambda x:np.log2(x) if x > 0 else np.nan)
    elif log == "log10":
        data = data.applymap(lambda x:np.log10(x) if x > 0 else np.nan)
    median = data.median(axis=1).sort_values(axis=0, ascending= True, na_position = 'first').to_frame()
    return median


def extractProteinSubjectRelationships(data):
    aux =  data.filter(regex = 'LFQ intensity')
    aux.columns = [c.split(" ")[2] for c in aux.columns]
    aux = aux.stack()
    aux = aux.reset_index()
    aux.columns = [':START_ID(Protein)', ':END_ID(Subject)', "LFQ intensity"]
    
    return aux

#ToDo
def extractModificationProteinSubjectRelationships(data):
    subjects =  data.filter(regex = 'LFQ intensity')
    subjects.columns = [c.split(" ")[2] for c in subjects.columns]
    subjects = subjects.stack()
    subjects = subjects.reset_index()
    icol = subjects.columns[0]
    subjects.columns  = [icol, "subject id","LFQ intensity"]
    
    modPosCol = proteomicsDataModPosCol
    cols = [c for c in data.columns if c in modPosCol]
    mod =  data.filter(cols)
    mod.columns = [modPosCol[c] for c in mod.columns]
      
    return mod

In [35]:
data = parseProteomicsDataset(projectID = 1, uri = "../Data/NAFLD/proteinGroups.txt")

In [81]:
data

,Glycation site positions,Gene names,Score,Q-value,LFQ intensity 22,LFQ intensity 23,LFQ intensity 20,LFQ intensity 21,LFQ intensity 26,LFQ intensity 27,...,Intensity 76,Intensity 77,LFQ intensity 17,LFQ intensity 16,LFQ intensity 15,LFQ intensity 14,LFQ intensity 13,LFQ intensity 12,LFQ intensity 11,LFQ intensity 10
Majority protein IDs,,,,,,,,,,,,,,,,,,,,,
P01911,NaN,HLA-DRB1,11.5300,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P05121,NaN,SERPINE1,83.1030,0.000000,NaN,NaN,NaN,NaN,6.966038,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P55083,NaN,MFAP4,272.8700,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P09972,NaN,ALDOC,114.8900,0.000000,NaN,NaN,NaN,7.938540,NaN,8.006552,...,7.986825,8.097951,NaN,NaN,7.666106,NaN,7.580788,NaN,NaN,NaN
Q96C19,NaN,EFHD2,3.8073,0.001086,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,5.571441,NaN,NaN,NaN,6.015402,NaN,NaN,NaN,NaN
P37198,NaN,NUP62,2.8480,0.002982,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
Q96A32,NaN,MYLPF,11.9280,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,6.381115,NaN,NaN,NaN
P55899,NaN,FCGRT,2.7262,0.003922,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
P35749,NaN,MYH11,151.1100,0.000000,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,8.007662,NaN,NaN,7.426544,NaN,NaN,NaN,NaN,NaN


In [76]:
extractModificationProteinSubjectRelationships(data)

,Glycation site positions
Majority protein IDs,
P01911,NaN
P05121,NaN
P55083,NaN
P09972,NaN
Q96C19,NaN
P37198,NaN
Q96A32,NaN
P55899,NaN
P35749,NaN


In [23]:
aux

,:START_ID(Protein),:END_ID(Subject),LFQ intensity
0,P01911,78,6.437164
1,P05121,26,6.966038
2,P09972,21,7.938540
3,P09972,27,8.006552
4,P09972,1,8.062657
5,P09972,7,7.916659
6,P09972,41,7.782902
7,P09972,46,8.062582
8,P09972,74,8.332438
9,P09972,77,8.065953
